In [21]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [22]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/MMR.jl")
include("./libs/X.jl")
include("./libs/nearly-equal.jl")

nearlyEqualLoose

In [23]:
# 評価基準数
num_criteria = [4];
# 真の区間重要度のセッティング
true_importance = ["A"]; # ["A", "B", "C", "D", "E"]

In [24]:
# 真の区間重要度を読み込み
for N in num_criteria
        
    for setting in true_importance
        df = CSV.File("./data/N"*string(N)*"_"*setting*"_Given_interval_weight.csv", header=false) |> DataFrame
        T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
        for i in 1:N
            push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
        end

        # 一対比較行列を1000個読み込み
        df1 = CSV.File("./data/N"*string(N)*"_a3_"*setting*"_PCM_int.csv", header=false) |> DataFrame

        # データフレームをN×Nのサブデータフレームに分割
        subdfs = split_dataframe(df1, N);

        # 1000個のデータで推定
        for (i, subdf) in enumerate(subdfs)

            # 一対比較行列を取り出す
            A = Matrix(subdf)
            # display(L"A = %$(matrixLaTeXString(A))")
            solution_m = MMR(A, EV)
            E_m = solution_m.W # 区間重要度の推定値
            distance_m = calculate_manhattan(interval_centers(T), interval_centers(E_m))/N
            solution_x = X(A, EV)
            E_x = solution_x.W # 区間重要度の推定値
            distance_x = calculate_manhattan(interval_centers(T), interval_centers(E_x))/N
            # println(distance_x, distance_m)

            if nearlyEqualLoose(distance_x, distance_m)
            else
                println(distance_x, distance_m)
                display(L"A = %$(matrixLaTeXString(A))")
            end

        end
    end
end



L"$A = \begin{pmatrix} 1.0 & 2.0 & 1.0 & 2.0 \\0.5 & 1.0 & 2.0 & 1.0 \\1.0 & 0.5 & 1.0 & 2.0 \\0.5 & 1.0 & 0.5 & 1.0 \\ \end{pmatrix}$"

0.04424399863214620.05524803921437184
